# Your Title Here

**Name(s)**: Ryan Wei, Eric Teng

**Website Link**: (your website link)

In [2]:
import pandas as pd
import numpy as np
import sys

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = 'plotly'

from lec_utils import * # Feel free to uncomment and use this. It'll make your plotly graphs look like ours in lecture!

## Step 1: Introduction

In [17]:
# TODO 
# are teams with VI or Jinx more likely to win??????????????????
games = pd.read_csv('2022.csv')

In [4]:
games.head()

,gameid,datacompleteness,url,league,...,deathsat25,opp_killsat25,opp_assistsat25,opp_deathsat25
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,...,1.0,0.0,2.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,...,2.0,1.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,...,0.0,3.0,4.0,3.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCKC,...,2.0,3.0,4.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCKC,...,2.0,0.0,7.0,2.0


## Step 2: Data Cleaning and Exploratory Data Analysis

In [5]:
# TODO: Data Clean
# for x in games.columns:
#     print(x)
# np.set_printoptions(threshold=sys.maxsize)
# print(list(games.columns))


# we need pick 1-5 and result
games['result'].unique()

array([0, 1])

In [6]:
# games.loc[:, ['pick1', 'pick2','pick3','pick4','pick5','result']]
# def func(x):
#     columns = ['pick1', 'pick2','pick3','pick4','pick5','result']
#     # print(x)
#     for col in columns:
#         if pd.isna(x[col]):
#             return None
#     return x totally unecessary

#change non vi/jinx games to nan to remove
games_edited = games.copy()
games_edited['pick1'] = games_edited['pick1'].transform(lambda x : np.where((x != "Jinx") & (x != "Vi"), pd.NA, x))
games_edited['pick2'] = games_edited['pick2'].transform(lambda x : np.where((x != "Jinx") & (x != "Vi"), pd.NA, x))
games_edited['pick3'] = games_edited['pick3'].transform(lambda x : np.where((x != "Jinx") & (x != "Vi"), pd.NA, x))
games_edited['pick4'] = games_edited['pick4'].transform(lambda x : np.where((x != "Jinx") & (x != "Vi"), pd.NA, x))
games_edited['pick5'] = games_edited['pick5'].transform(lambda x : np.where((x != "Jinx") & (x != "Vi"), pd.NA, x))

#remove all games with nan values or no vi/jinx
games_edited = games_edited.dropna(how='all', subset=['pick1','pick2','pick3','pick4','pick5'])
games_edited = games_edited.dropna(subset=['result'])


games_edited[['pick1','pick2','pick3','pick4','pick5','result']]

,pick1,pick2,pick3,pick4,pick5,result
11,Jinx,<NA>,<NA>,<NA>,<NA>,1
34,Jinx,<NA>,<NA>,<NA>,<NA>,1
58,Jinx,<NA>,<NA>,<NA>,<NA>,1
...,...,...,...,...,...,...
150562,<NA>,Vi,<NA>,<NA>,<NA>,1
150575,<NA>,Vi,<NA>,<NA>,<NA>,0
150587,<NA>,Vi,<NA>,<NA>,<NA>,1


In [10]:
# Turn into one-hot encoding
one_hot = pd.DataFrame()
one_hot["Jinx"] = games_edited.eq("Jinx").any(axis=1)
one_hot["Vi"] = games_edited.eq("Vi").any(axis=1)
one_hot["Result"] = games_edited["result"]
one_hot.head(10)

,Jinx,Vi,Result
11,True,False,1
34,True,False,1
58,True,False,1
...,...,...,...
166,True,False,1
191,True,False,0
202,True,False,0


In [7]:
vi_only_games = one_hot[(one_hot["Vi"] == True) & (one_hot["Jinx"] == False)].shape[0]
jinx_only_games = one_hot[(one_hot["Jinx"] == True) & (one_hot["Vi"] == False)].shape[0]

vi_only_win = one_hot[(one_hot["Vi"] == True) & (one_hot["Jinx"] == False) & (one_hot['Result'] == 1)].shape[0]
jinx_only_win= one_hot[(one_hot["Jinx"] == True) & (one_hot["Vi"] == False) & (one_hot['Result'] == 1)].shape[0]

print(vi_only_games, jinx_only_games, vi_only_win, jinx_only_win)

512 3292 236 1644


In [14]:
# graph
# games w/ jinx or vi and games w/o, one for each

#jinx
# graph
# games w/ jinx or vi and games w/o, one for each

# fig = go.Figure()
# fig.add_trace(go.Bar(x=["Total Games", "Jinx Games"], y=[games.shape[0], games['jinx'].sum()], barmode='stack'))
# fig.update_layout(barmode='stack')
# fig.show()

games['jinx'] = games.eq("Jinx").any(axis=1)

games['vi'] = games.eq("Vi").any(axis=1)

summary = pd.DataFrame({
    'Champion': ['Jinx', 'Vi'],
    'Games without Vi or Jinx': [len(games) - games['jinx'].sum(), len(games) - games['vi'].sum()] ,
    'Games with Vi or Jinx': [games['jinx'].sum(), games['vi'].sum()]
})

# Convert to long format for Plotly Express
df_long = summary.melt(id_vars='Champion', var_name='Type', value_name='Count')
df_long['Type'] = pd.Categorical(df_long['Type'], categories=['Games with Vi or Jinx', 'Games without Vi or Jinx'], ordered=True)
fig = px.bar(df_long, x='Champion', y='Count', color='Type', barmode='stack')
# fig = px.bar(df_long, x='Champion', y='Count', color='Type', barmode='stack')
# fig = px.bar(games, x=["Total Games", "Jinx Games"], y=[games.shape[0], games['jinx'].sum()], color=["Total Games", "Jinx Games"], barmode='stack')
# fig.update_layout(barmode='stack')
fig.show()

In [29]:
# TODO: Analysis, univariate/bivariate
#Jinx Win rate
games[['teamname','pick1', 'pick2','pick3','pick4','pick5', 'result']]
jinxwins = games.copy()
filtered_jinxwins = jinxwins[jinxwins[['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].notna().all(axis=1)]
filtered_jinxwins = filtered_jinxwins[['teamname','pick1', 'pick2','pick3','pick4','pick5', 'result']]

In [48]:
def hasJinx(row):
    return any(row[['pick1', 'pick2', 'pick3', 'pick4', 'pick5']] == 'Jinx')

filtered_jinxwins = filtered_jinxwins[filtered_jinxwins.apply(hasJinx, axis=1)]
team_jinx_games = filtered_jinxwins.groupby('teamname').size()
team_jinx_wins = filtered_jinxwins[filtered_jinxwins['result'] == 1].groupby('teamname').size()

jinxwinrate = (team_jinx_wins / team_jinx_games).reset_index().rename(columns={0 : "jinxwinrate"})
fig = px.histogram(jinxwinrate, x="jinxwinrate",nbins=15)
fig.show()

In [36]:
viwins = games.copy()
filtered_viwins = viwins[viwins[['pick1', 'pick2', 'pick3', 'pick4', 'pick5']].notna().all(axis=1)]
filtered_viwins = filtered_viwins[['teamname','pick1', 'pick2','pick3','pick4','pick5', 'result']]

In [ ]:
def hasVi(row):
    return any(row[['pick1', 'pick2', 'pick3', 'pick4', 'pick5']] == 'Vi')

filtered_viwins = filtered_viwins[filtered_viwins.apply(hasVi, axis=1)]
team_vi_games = filtered_viwins.groupby('teamname').size()
team_vi_wins = filtered_viwins[filtered_viwins['result'] == 1].groupby('teamname').size()

viwinrate = (team_vi_wins / team_vi_games).reset_index().rename(columns={0 : "viwinrate"})
fig = px.histogram(viwinrate, x="viwinrate",nbins=15, color_discrete_sequence=["#870146"])
fig.show()

(197, 2)


In [59]:
df1 = pd.DataFrame({'winrate' : jinxwinrate['jinxwinrate'], 'category' : 'Jinx'})
df2 = pd.DataFrame({'winrate' : viwinrate['viwinrate'], 'category' : 'Vi'})
viandjinxwinrates = pd.concat([df1, df2])
fig = px.box(viandjinxwinrates, x="category", y="winrate",color="category", color_discrete_map={'Vi' : '#870146', 'Jinx' : '#324AB2'})
fig.show()

In [ ]:
# TODO: Aggregates

In [ ]:
#TODO: Imputation

## Step 3: Framing a Prediction Problem

In [ ]:
# TOD

## Step 4: Baseline Model

In [ ]:
# TODO

## Step 5: Final Model

In [ ]:
# TODO